<a href="https://colab.research.google.com/github/zcongfly/huggingface-nlp-learning-note/blob/main/03_%E6%A8%A1%E5%9E%8B_(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模型 (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

AutoModel类及其所有相关项实际上是对库中各种可用模型的简单包装。它是一个聪明的包装器，因为它可以自动猜测检查点的适当模型体系结构，然后用该体系结构实例化模型。

但是，如果您知道要使用的模型类型，则可以使用直接定义其体系结构的类。让我们看看这是如何与BERT模型一起工作的。

## 创建transformer

初始化BERT模型需要做的第一件事是加载一个配置对象：

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

该配置包含许多用于构建模型的属性：

In [ ]:
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



虽然您还没有看到所有这些属性的作用，但您应该认识其中的一些属性： hidden_size 属性定义了 hidden_states 向量的大小， num_hidden_layers 定义了 Transformer 模型的层数。

### 不同的加载方式

从默认配置创建模型会使用随机值对其进行初始化：

In [ ]:
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

# Model is randomly initialized!

In [ ]:
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



该状态下模型可以使用，但会输出乱码；它需要先接受培训。我们可以根据手头的任务从头开始训练模型，但正如您在第 1 章中看到的那样，这需要很长时间和大量数据，并且会对环境产生不可忽视的影响。为了避免不必要和重复的工作，必须能够共享和重用已经训练过的模型。

加载一个已经训练好的 Transformer 模型很简单——我们可以使用 from_pretrained() 方法来做到这一点：

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

正如您之前看到的，我们可以用等效的 AutoModel 类替换 BertModel 。我们将从现在开始这样做，因为这会产生与检查点无关的代码；如果您的代码适用于一个检查点，那么它应该可以与另一个检查点无缝协作。即使架构不同，这也适用，只要检查点是为类似任务（例如，情感分析任务）训练的。

在上面的代码示例中，我们没有使用 BertConfig ，而是通过 bert-base-cased 标识符加载了一个预训练模型。这是BERT作者自己训练的模型checkpoint；您可以在其[模型卡](https://huggingface.co/bert-base-cased)中找到有关它的更多详细信息。

该模型现在已使用检查点的所有权重进行了初始化。它可以直接用于推断它所训练的任务，也可以在新任务上进行微调。通过使用预训练的权重而不是从头开始训练，我们可以快速取得良好的效果。

权重已下载并缓存（因此以后调用 from_pretrained() 方法不会重新下载它们）在缓存文件夹中，默认为 ~/.cache/huggingface/transformers。您可以通过设置 HF_HOME 环境变量来自定义缓存文件夹。

用于加载模型的标识符可以是 Model Hub 上任何模型的标识符，只要它与 BERT 架构兼容即可。可以在[此处](https://huggingface.co/models?other=bert)找到可用 BERT 检查点的完整列表。

### 保存模型

保存模型和加载模型一样简单——我们使用 save_pretrained() 方法，它类似于 from_pretrained() 方法：

In [ ]:
model.save_pretrained("directory_on_my_computer")

这会将两个文件保存到您的磁盘：

In [ ]:
!ls directory_on_my_computer

config.json  pytorch_model.bin


In [ ]:
!ls /content/directory_on_my_computer

config.json  pytorch_model.bin


如果你看一下 config.json 文件，你就会认识到构建模型架构所必需的属性。该文件还包含一些元数据，例如检查点的来源以及您上次保存检查点时使用的Transformers 版本。

pytorch_model.bin 文件被称为状态字典；它包含您模型的所有权重。这两个文件齐头并进；配置是了解模型架构所必需的，而模型权重是模型的参数。

如果您使用的是谷歌的colab，可以通过左侧的“文件”目录找到对应的文件，右键点击下载即可。

也可以通过`google.colab.files`模块将需要的配置文件下载到本地，具体代码示例如下：

In [ ]:
from google.colab import files

# 指定要下载的文件路径
file_path = "/content/directory_on_my_computer/config.json"

# 下载文件
files.download(file_path)

`google.colab.files`模块不能直接下载一个目录下的所有文件，如果文件较多，可以先执行`zip -r model_files.zip /content/directory_on_my_computer`命令，将`/content/directory_on_my_computer`目录下的文件打包成zip格式保存在当前工作目录中，然后通过files模块下载该压缩包即可。

In [ ]:
# 该命令会将directory_on_my_computer文件夹下的文件打包成model_files.zip并存储在当前工作目录/content中
!zip -r model_files.zip /content/directory_on_my_computer

updating: content/directory_on_my_computer/ (stored 0%)
updating: content/directory_on_my_computer/pytorch_model.bin (deflated 7%)
updating: content/directory_on_my_computer/config.json (deflated 48%)


In [ ]:
!ls

directory_on_my_computer  model_files.zip  sample_data


In [ ]:
files.download("model_files.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 使用 Transformer 模型进行推理

现在您知道如何加载和保存模型，让我们尝试使用它来进行一些预测。 Transformer 模型只能处理数字（分词器生成的数字）。但在我们讨论分词器之前，让我们先探讨一下模型接受的输入。

Tokenizers 可以负责将输入转换为适当框架的张量，但为了帮助您了解正在发生的事情，我们将快速了解一下在将输入发送到模型之前必须完成的工作。

假设我们有几个序列：

In [ ]:
sequences = ["Hello!", "Cool.", "Nice!"]

分词器将这些转换为通常称为输入 ID 的词汇表索引。每个序列现在都是一个数字列表！结果输出是：

In [ ]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

使用torch框架把encoded_sequences转换成张量：

In [ ]:
import torch

model_inputs = torch.tensor(encoded_sequences)

## 使用张量作为模型的输入

在模型中使用张量非常简单——我们只需使用输入调用模型：

In [ ]:
output = model(model_inputs)

虽然模型接受许多不同的参数，但只有输入 ID 是必需的。我们将解释其他参数的作用以及稍后需要它们的时间，但首先我们需要仔细研究构建 Transformer 模型可以理解的输入的分词器Tokenizer。